# 특정 도로 다운로드

In [28]:
# Modules
import requests
import urllib.request
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import re

from datetime import datetime
import os
import time

In [7]:
def authentication(auth_key, road_type):
    if road_type == 'ex':
        url = 'http://openapi.its.go.kr:8081/api/NCCTVInfo?key={}&ReqType=2&MinX=120&MaxX=135&MinY=30&MaxY=45&type=ex'.format(auth_key) # 고속도로 url
    else:
        url = 'http://openapi.its.go.kr:8081/api/NCCTVInfo?key={}&ReqType=2&MinX=120&MaxX=135&MinY=30&MaxY=45&type=its'.format(auth_key) # 국도 url
    return url

In [8]:
def request_api(url):
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')

    # cctv 이름 / url / 좌표 데이터프레임
    name = [i.text for i in soup.findAll("cctvname")]
    url = [i.text for i in soup.findAll("cctvurl")]
    coordx = [i.text for i in soup.findAll("coordx")]
    coordy = [i.text for i in soup.findAll("coordy")]

    cctv_df = pd.DataFrame(data={'name':name, 'url':url, 'coordx':coordx, 'coordy':coordy})
    return cctv_df

In [ ]:
def format_name(cctv_name):
    p = re.compile('[\w\[\]]+')
    regex_name = re.findall(p, cctv_name)
    formatted_name = ''
    for i in range(len(regex_name)):
        formatted_name += regex_name[i]

    now = datetime.now()
    formatted_name = '{}_{}_{}'.format(formatted_name, now.strftime('%m%d'), now.strftime('%H%M'))
    return formatted_name

In [9]:
auth_key = '1560428298105'
road_type = 'ex'
url = authentication(auth_key, road_type)
cctv_df = request_api(url)

In [26]:
abnormal_name = '창원분기점'
abnormal_cctv = cctv_df[cctv_df['name'].str.contains(abnormal_name)]
abnormal_url = abnormal_cctv.iloc[0].loc['url']

In [33]:
save_path = 'C:/users/yoon/desktop/temp/이상주행/normal/'
download_num = 10
sleep_time = 300

In [ ]:
for i in range(download_num):
    urllib.request.urlretrieve(abnormal_url, os.path.join(save_path, '{}_{}.mp4'.format(abnormal_name, i)))
    print(i, ' 다운로드 완료')
    time.sleep(sleep_time)

0  다운로드 완료
1  다운로드 완료
2  다운로드 완료
3  다운로드 완료
4  다운로드 완료
5  다운로드 완료
